## Detect Roses and Weeds in a Video

This section uses our trained YOLOv8 model to run inference on a video (`.mov` or `.mp4`).
It will:
1. Run YOLOv8 detection on each frame.
2. Save an annotated video with bounding boxes and labels.
3. Convert the YOLO output `.avi` to a more friendly format such as `.mp4`

You can use this to visualize how well the model performs on real footage.


# 1. Imports and paths

In [6]:
from ultralytics import YOLO
from pathlib import Path
import subprocess

PROJECT_ROOT = Path(r"D:/Ai Systems Group")
MODEL_PATH   = PROJECT_ROOT / "work_dirs/yolov8s-seg-weeds/weights/best.pt"
VIDEO_PATH   = PROJECT_ROOT / "data/weeds_yolo/video/rose_row.mp4"
OUT_DIR      = PROJECT_ROOT / "model_predictions"
VIDEO_PATH = Path(VIDEO_PATH)

MODEL_PATH.exists(), VIDEO_PATH.exists()


(True, True)

# 2. Run interference

In [7]:
model = YOLO(str(MODEL_PATH))

results = model.predict(
    source=str(VIDEO_PATH),
    conf=0.30,           # confidence threshold
    imgsz=1024,          # match your training resolution
    device=0,            # GPU (set to 'cpu' if no GPU)
    save=True,           # write the annotated video
    project=str(OUT_DIR),
    name="video_prediction",
    vid_stride=1,        # process every frame (increase to 2+ for speed)
    classes=[0, 1],      # detect roses and weeds (adjust if needed)
)

print(" Saved to:", OUT_DIR / "video_prediction")



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/765) D:\Ai Systems Group\data\weeds_yolo\video\rose_row.mp4: 1024x576 16 weeds, 3 roses, 783.0ms
video 1/1 (frame 2/765) D:\Ai Systems Group\data\weeds_yolo\video\rose_row.mp4: 1024x576 18 weeds, 4 roses, 872.7ms
video 1/1 (frame 3/765) D:\Ai Systems Group\data\weeds_yolo\video\rose_row.mp4: 1024x576 17 weeds, 4 roses, 857.4ms
video 1/1 (frame 4/765) D:\Ai Systems Group\data\weeds_yolo\video\rose_row.mp4: 1024x576 18 weeds, 2 roses, 801

OutOfMemoryError: CUDA out of memory. Tried to allocate 182.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 13.81 GiB is allocated by PyTorch, and 435.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 3. Convert YOLO output `.avi` to `.mp4`

YOLOv8 saves videos as `.avi` by default for compatibility.
This cell converts your `.avi` result to a high-quality `.mp4` using H.264 encoding.
The conversion is **lossless** (no quality degradation).


In [9]:
import cv2
from pathlib import Path

SOURCE_PATH = Path(r"/model_predictions/video_prediction2/rose_row.avi")
DESTINATION_PATH = SOURCE_PATH.with_suffix(".mp4")

cap = cv2.VideoCapture(str(SOURCE_PATH))
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(str(DESTINATION_PATH), fourcc, float(fps), (w, h))

while True:
    ok, frame = cap.read()
    if not ok:
        break
    out.write(frame)

cap.release()
out.release()
print(f"Conversion complete: {DESTINATION_PATH}")

Conversion complete: D:\Ai Systems Group\model_predictions\video_prediction3\rose_row.mp4
